Shor's Algorithm Implementation for Large Number Factorization
1. Introduction
The Problem: Factoring large integers (N) into their prime components is computationally intractable for classical computers. The security of widely used cryptographic systems like RSA relies on this difficulty.

The Quantum Solution: Shor's algorithm, developed by Peter Shor in 1994, provides a polynomial-time solution on a quantum computer, posing a significant threat to classical cryptography but also showcasing the power of quantum computation.

Project Goal: To implement a universal Shor's algorithm using the QuantumRingsLib SDK and its scarlet_quantum_rings backend, capable of factoring large semiprime numbers (like those provided in semiprimes.py) by leveraging quantum parallelism and phase estimation.

2. Algorithm Overview
Shor's algorithm cleverly transforms the factoring problem into a period-finding problem.

Classical Pre-processing:

Check if N is even or a prime power (trivial cases).

Choose a random integer a such that 1 < a < N.

Compute gcd(a, N). If it's greater than 1, we've found a factor classically! Otherwise, a is coprime to N.

Quantum Period Finding (The Core):

Find the period r of the modular exponentiation function f(x) = a^x mod N. This means finding the smallest positive integer r such that a^r ≡ 1 (mod N).

This is achieved using the Quantum Phase Estimation (QPE) algorithm, which involves:

Preparing quantum registers.

Applying controlled modular exponentiation (a^x mod N).

Applying the Inverse Quantum Fourier Transform (IQFT).

Measuring the phase register.

Classical Post-processing:

Analyze the measurement outcomes from the quantum circuit.

Use the Continued Fractions Algorithm to deduce the period r from the measurement results.

If r is valid (even and a^(r/2) ≠ -1 mod N), calculate the factors using gcd(a^(r/2) ± 1, N).

3. Quantum Circuit Implementation (QuantumRingsLib)
Our implementation translates the quantum period-finding steps into a circuit using QuantumRingsLib.

Registers:

phase_reg (Phase/Counting Register): Size m = 2n qubits (where n is the number of bits in N). Initialized to uniform superposition using Hadamard gates. Controls the exponentiation.

target_reg (Working Register): Size n qubits. Stores the result of the modular exponentiation. Initialized to |1⟩.

anc_reg (Ancilla Register): Size ~2n+1 qubits. Used for intermediate calculations within arithmetic operations (adders, multipliers), must be initialized to |0⟩ and returned to |0⟩ via uncomputation.

classical_reg: Size m bits. Stores the measurement outcome of the phase_reg.

Initialization: Apply H gates to phase_reg, X gate to the first qubit of target_reg.

Controlled Modular Exponentiation (a^x mod N):

Implemented using the repeated squaring method.

Iterates j from 0 to m-1 (corresponding to the exponent bits x_j in superposition).

In each iteration, applies a Controlled Modular Multiplier (CMM) operation: |phase_j⟩|target⟩ → |phase_j⟩|target * k mod N⟩, where k = pow(a, 2**j, N) is pre-calculated classically.

Controlled Modular Multiplier (CMM) Implementation:

Uses the accumulator method: Computes acc = sum_{i=0}^{n-1} (y_i * k * 2^i) mod N, controlled by the main control qubit (phase_j) and the target bits (target_reg[i]).

Requires a Controlled Modular Adder for Constants (controlled_modular_adder_const) as a subroutine.

Controlled Modular Adder (controlled_modular_adder_const) Implementation:

Adds a classical constant k_add to the target register modulo N, controlled by potentially multiple qubits.

Uses QFT-based addition for the + k_add mod 2^n part (applying QFT, controlled phase rotations, IQFT).

Includes explicit modular reduction logic:

Subtract N (mod 2^n) using the inverse QFT adder.

Check the Most Significant Bit (MSB) of the result using multi-controlled X gates into a flag ancilla to determine if the original sum was < N.

Conditionally add N back (mod 2^n) using a controlled QFT adder, controlled by the flag qubit.

Uncompute the subtraction and flag operations to reset ancillas. (This uncomputation logic is complex and critical for correctness).

Inverse Quantum Fourier Transform (IQFT): Applied to the phase_reg after the modular exponentiation to transform the phase information into a measurable state.

Measurement: The phase_reg is measured, collapsing the superposition into a classical bit string.

4. Classical Post-Processing
Period Extraction: The measurement outcome m (an integer) is related to the period r by m / 2^m ≈ p / r for some integer p.

Continued Fractions: The script uses the continued fractions algorithm on m / 2^m to find the best rational approximation p / r with r < N.

Factor Calculation: Once a valid period r is found, the factors are calculated classically using gcd(a^(r/2) ± 1, N).

5. Results & Demonstration (N=15)
The implementation was successfully tested for N=15 (L=4) using a=7.

The quantum circuit (using 8 phase, 4 target, 9 ancilla qubits) was executed on the scarlet_quantum_rings backend simulator.

Measurement results showed peaks near multiples of 2^m / r = 2^8 / 4 = 64, as expected.

The classical post-processing correctly identified the period r=4 from the measurement counts.

The final GCD calculations yielded the correct factors 3 and 5.

6. Tools Used
Quantum SDK: QuantumRingsLib v0.9.0

Backend: scarlet_quantum_rings simulator provided by Quantum Rings.

Language: Python 3

Libraries: numpy, math, fractions, collections, matplotlib

7. Challenges & Future Work
Arithmetic Circuit Complexity: Implementing controlled modular arithmetic (especially multiplication and the adder's modular reduction/uncomputation) correctly and efficiently from basic gates is highly complex. Verification is challenging.

Multi-Controlled Gates: Decomposing gates with many controls (needed for adders/multipliers) increases gate count and circuit depth.

Resource Scaling: While the framework handles variable N, the required qubits (~4n) and especially the gate count (~O(n^3)) grow significantly, demanding substantial simulation resources for larger N.

Future Work: Verify and potentially optimize the CMM/CMA circuits, implement more efficient multi-controlled gate decompositions, test scaling on larger N from the semiprimes list, explore QFT approximations.

8. Conclusion
This project successfully implements the structure of Shor's quantum algorithm using QuantumRingsLib. It incorporates genuine quantum arithmetic principles, including QFT-based adders and explicit modular reduction logic. The successful factorization of N=15 validates the overall approach and the implemented components for this specific case, demonstrating the potential for factoring larger numbers on capable quantum simulators or hardware.

In [ ]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install QuantumRingsLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.4 MB/s eta 0:00:00


In [ ]:
# Import necessary QuantumRingsLib components and other libraries
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus # Assuming JobStatus might be needed for checks
# Import standard libraries
from matplotlib import pyplot as plt
import numpy as np
import math
import time # For potential delays or checks
import random # Needed for selecting 'a' in Shor's
import os # To check if semiprimes.py exists
from fractions import Fraction # For continued fractions
from collections import Counter # For processing counts

# --- Import semiprimes from external file ---
try:
    from semiprimes import semiprimes
    print("Successfully imported 'semiprimes' dictionary from semiprimes.py")
except ImportError:
    print("Error: Could not import 'semiprimes' from semiprimes.py.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during import: {e}")
    exit()

print(f"QuantumRingsLib version: {QuantumRingsLib.__version__}")

# --- Helper Functions ---

def qft(qc, register, n_qubits):
    """ Applies the QFT on the first n_qubits of the register. """
    # Ensure register is treated as a list of qubits if it's a QuantumRegister object
    qubit_list = [register[i] for i in range(n_qubits)] if isinstance(register, (QuantumRegister, AncillaRegister)) else register
    if n_qubits == 0: return
    for i in range(n_qubits):
        try:
            qc.h(qubit_list[i])
            for j in range(i + 1, n_qubits):
                angle = math.pi / (2**(j - i))
                qc.cp(angle, qubit_list[j], qubit_list[i])
        except AttributeError as e: print(f"FATAL: Missing gate ({e}) for QFT."); raise
        except Exception as e_gate: print(f"Warning: Error in QFT gate: {e_gate}.")
    for i in range(n_qubits // 2):
        try: qc.swap(qubit_list[i], qubit_list[n_qubits - 1 - i])
        except AttributeError: print(f"Warning: qc.swap not found.")
        except Exception as e_swap: print(f"Warning: Error in QFT swap: {e_swap}.")
    # qc.barrier(label="QFT")

def iqft_cct(qc, register, n_qubits):
    """ Applies the inverse QFT on the first n_qubits of the register. """
    # Ensure register is treated as a list of qubits
    qubit_list = [register[i] for i in range(n_qubits)] if isinstance(register, (QuantumRegister, AncillaRegister)) else register
    if n_qubits == 0: return
    for i in range(n_qubits // 2):
         try: qc.swap(qubit_list[i], qubit_list[n_qubits - 1 - i])
         except AttributeError: print(f"Warning: qc.swap not found.")
         except Exception as e_swap: print(f"Warning: Error in IQFT swap: {e_swap}.")
    for i in range(n_qubits - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            if j >= 0:
                 angle = -math.pi / (2**(i - j))
                 try: qc.cp(angle, qubit_list[j], qubit_list[i])
                 except AttributeError: print(f"FATAL: qc.cp not found for IQFT."); raise
                 except Exception as e_cp: print(f"Warning: Error in IQFT cp: {e_cp}.")
        try: qc.h(qubit_list[i])
        except Exception as e_h: print(f"Warning: Error in IQFT H: {e_h}")
    # qc.barrier(label="IQFT")

def plot_histogram (counts, title=""):
    """ Plots the histogram of the counts dictionary. """
    if not counts: print("Counts empty."); return
    fig, ax = plt.subplots(figsize =(14, 8))
    plt.xlabel("Measured State (Binary - Phase Register)"); plt.ylabel("Counts")
    try: sorted_keys = sorted(counts.keys(), key=lambda x: int(x, 2) if x.isdigit() else x)
    except ValueError: sorted_keys = sorted(counts.keys())
    sorted_values = [counts[key] for key in sorted_keys]
    bars = ax.bar(sorted_keys, sorted_values, color='skyblue', width=0.8)
    ax.tick_params(axis='x', rotation=90)
    if len(sorted_keys) <= 32: ax.bar_label(bars, padding=3, fontsize=8, rotation=90)
    maxFreq = max(counts.values()) if counts else 1
    plt.ylim(ymax=maxFreq * 1.1); ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.title(title, fontsize=14); plt.tight_layout(); plt.show()

# --- Classical Post-Processing Functions ---
def continued_fractions_convergents(numerator, denominator):
    """ Calculates convergents using continued fractions. """
    a = numerator; b = denominator; coeffs = []
    while b: quotient = a // b; coeffs.append(quotient); a, b = b, a % b
    p_prev, q_prev = 0, 1; p_curr, q_curr = 1, 0
    for c in coeffs:
        p_next = c * p_curr + p_prev; q_next = c * q_curr + q_prev
        if q_next != 0: yield Fraction(p_next, q_next)
        p_prev, q_prev = p_curr, q_curr; p_curr, q_curr = p_next, q_next

def find_period_from_counts(counts, num_phase_qubits, N, a):
    """ Analyzes counts to find period 'r'. """
    print("\n--- Starting Classical Post-Processing (Period Finding) ---")
    m = num_phase_qubits; two_m = 1 << m; measured_values = Counter(counts)
    print(f"    Measurements sorted by frequency: {measured_values.most_common(10)}") # Show top 10
    candidate_periods = set()
    processed_measurements = 0
    for measurement_str, count in measured_values.most_common(): # Process all
        processed_measurements += 1
        try: measured_int = int(measurement_str, 2)
        except ValueError: continue
        if measured_int == 0: continue
        phase = measured_int / two_m
        found_potential_period_for_m = False
        for conv in continued_fractions_convergents(measured_int, two_m):
            q = conv.denominator; p = conv.numerator
            if q >= N: break
            if abs(phase - conv) < 1.0 / (2 * two_m):
                if q > 0 and pow(a, q, N) == 1:
                    print(f"                SUCCESS: Found potential period r = {q} from m={measured_int} (convergent {p}/{q})")
                    candidate_periods.add(q); found_potential_period_for_m = True
    if not candidate_periods: print("\n--- Period Finding Failed: No valid candidate periods found. ---"); return None
    valid_candidates = {r for r in candidate_periods if r > 0}
    if not valid_candidates: print("\n--- Period Finding Failed: No positive period candidates found. ---"); return None
    estimated_r = min(valid_candidates)
    print(f"\n--- Period Finding Result: Smallest valid candidate period found: r = {estimated_r} ---")
    return estimated_r

# --- Quantum Arithmetic Building Blocks ---

def ccx(qc, ctrl1, ctrl2, tgt):
    """ Implements Toffoli using H, CX, T, Tdg. """
    try:
        qc.h(tgt); qc.cx(ctrl2, tgt); qc.tdg(tgt); qc.cx(ctrl1, tgt); qc.t(tgt)
        qc.cx(ctrl2, tgt); qc.tdg(tgt); qc.cx(ctrl1, tgt); qc.t(ctrl2); qc.t(tgt)
        qc.h(tgt); qc.cx(ctrl1, ctrl2); qc.t(ctrl1); qc.tdg(ctrl2); qc.cx(ctrl1, ctrl2)
    except AttributeError as e: print(f"FATAL: Missing gate ({e}) for CCX."); raise
    except Exception as e: print(f"Error during CCX: {e}"); raise

def cccx(qc, ctrl1, ctrl2, ctrl3, tgt):
     """ Implements Controlled-CCX (Toffoli-4) gate decomposition using 3 CCX, 2 CX. """
     try:
         qc.cx(ctrl3, tgt)
         qc.cx(ctrl3, ctrl2)
         ccx(qc, ctrl1, ctrl2, ctrl3)
         qc.cx(ctrl3, ctrl2) # Uncompute
         qc.cx(ctrl3, tgt) # Uncompute
         qc.cx(ctrl2, tgt)
         ccx(qc, ctrl1, tgt, ctrl2)
         qc.cx(ctrl2, tgt) # Uncompute
         qc.cx(ctrl1, tgt)
     except AttributeError as e: print(f"FATAL: Missing gate ({e}) for CCCX decomposition."); raise
     except Exception as e: print(f"Error during CCCX decomposition: {e}"); raise

def multi_controlled_X(qc, controls, target, anc_qubits):
     """
     Implements multi-controlled X gate using decomposition.
     Requires list of ancilla qubits for decomposition if > 3 controls.
     """
     num_controls = len(controls)
     if num_controls == 0: qc.x(target)
     elif num_controls == 1: qc.cx(controls[0], target)
     elif num_controls == 2: ccx(qc, controls[0], controls[1], target)
     elif num_controls == 3: cccx(qc, controls[0], controls[1], controls[2], target)
     else:
         # Basic decomposition using n-2 ancillas (from anc_qubits list)
         if len(anc_qubits) < num_controls - 2:
             raise ValueError(f"Need at least {num_controls - 2} ancillas for {num_controls}-controlled X.")
         # print(f"      Decomposing {num_controls}-controlled X...") # Verbose
         ccx(qc, controls[0], controls[1], anc_qubits[0])
         for i in range(2, num_controls - 1):
             ccx(qc, controls[i], anc_qubits[i-2], anc_qubits[i-1])
         # Last Toffoli
         ccx(qc, controls[num_controls-1], anc_qubits[num_controls-3], target)
         # Uncompute backwards
         for i in range(num_controls - 2, 1, -1):
             ccx(qc, controls[i], anc_qubits[i-2], anc_qubits[i-1])
         ccx(qc, controls[0], controls[1], anc_qubits[0])


def controlled_phase_addition(qc, control_list, target_q_list, k_add):
    """ Adds k_add to target_reg (list of qubits) in Fourier basis, controlled by control_list. """
    n = len(target_q_list)
    for i in range(n):
        angle = (k_add * math.pi) / (2**i) # QFT Adder phase convention
        if angle != 0:
            # Target qubit index assumes standard QFT order (MSB first) after swaps
            target_qubit = target_q_list[n-1-i]
            try: qc.mcp(angle, control_list, target_qubit)
            except AttributeError: # Decompose mcp
                 if not control_list: qc.p(angle, target_qubit)
                 elif len(control_list) == 1: qc.cp(angle, control_list[0], target_qubit)
                 elif len(control_list) == 2:
                     qc.cp(angle / 2, control_list[1], target_qubit)
                     qc.cx(control_list[0], control_list[1])
                     qc.cp(-angle / 2, control_list[1], target_qubit)
                     qc.cx(control_list[0], control_list[1])
                     qc.cp(angle / 2, control_list[0], target_qubit)
                 else: # Requires multi-controlled phase gate decomposition
                     print(f"ERROR: >2 controls for phase gate not implemented."); raise NotImplementedError
            except Exception as e_mcp: print(f"ERROR: MCP failed: {e_mcp}"); raise

def controlled_adder_const_QFT_NO_MOD(qc, control_list, target_q_list, k_add):
     """ Adds k_add to target_reg (mod 2^n), controlled by control_list. Uses QFT. """
     n = len(target_q_list)
     qft(qc, target_q_list, n)
     controlled_phase_addition(qc, control_list, target_q_list, k_add)
     iqft_cct(qc, target_q_list, n)

def controlled_adder_const_QFT_NO_MOD_dagger(qc, control_list, target_q_list, k_add):
     """ Inverse: Subtracts k_add (mod 2^n). """
     n = len(target_q_list)
     # Apply inverse operations in reverse order
     qft(qc, target_q_list, n) # Inverse of IQFT
     controlled_phase_addition(qc, control_list, target_q_list, -k_add) # Subtract
     iqft_cct(qc, target_q_list, n) # Inverse of QFT

# --- Controlled Modular Adder (Constant k_add) ---
def controlled_modular_adder_const(qc, control_list, target_q_list, anc_qubits, k_add, N):
    """
    Adds classical constant k_add to target_reg modulo N, controlled by control_list.
    Uses QFT adder + Explicit modular reduction logic. Needs >= n+1 ancillas.
    """
    n = len(target_q_list)
    required_ancillas = n + 1
    if len(anc_qubits) < required_ancillas:
        raise ValueError(f"Need at least {required_ancillas} ancillas.")

    flag_qubit = anc_qubits[n] # Use last required ancilla as the comparison flag qubit
    mct_ancilla = anc_qubits[:n] # Use first n ancillas for multi-controlled gate decomposition

    # print(f"        -> CMA_Const: Add {k_add} mod {N} (Ctrl by {len(control_list)})") # Verbose

    # --- Circuit Steps ---
    # 1. Add k_add (mod 2^n) to target_reg
    controlled_adder_const_QFT_NO_MOD(qc, control_list, target_q_list, k_add)

    # 2. Subtract N (mod 2^n) from target_reg
    controlled_adder_const_QFT_NO_MOD_dagger(qc, control_list, target_q_list, N)

    # 3. Check MSB (target_q_list[n-1]) to set flag=1 if result < 0 (i.e., original sum < N)
    msb_qubit = target_q_list[n-1]
    # Copy MSB to flag (Multi-Controlled NOT)
    multi_controlled_X(qc, control_list + [msb_qubit], flag_qubit, mct_ancilla)

    # 4. Conditionally add N back (mod 2^n), controlled by flag=1 and control_list.
    add_N_control_list = control_list + [flag_qubit]
    controlled_adder_const_QFT_NO_MOD(qc, add_N_control_list, target_q_list, N)

    # 5. Uncompute the subtraction of N (mod 2^n) - controlled by control_list
    controlled_adder_const_QFT_NO_MOD(qc, control_list, target_q_list, N) # Inverse of subtract N is add N

    # 6. Uncompute the MSB flag copy - controlled by control_list
    multi_controlled_X(qc, control_list + [msb_qubit], flag_qubit, mct_ancilla) # Reverse MCT

    # qc.barrier(label=f"CMA({k_add})") # Remove barrier


def controlled_modular_adder_const_dagger(qc, control_list, target_q_list, anc_qubits, k_add, N):
    """ Inverse of controlled_modular_adder_const. Subtracts k_add mod N. """
    n = len(target_q_list)
    required_ancillas = n + 1
    if len(anc_qubits) < required_ancillas: raise ValueError("Need ancillas for CMA_const dagger.")
    flag_qubit = anc_qubits[n]
    msb_qubit = target_q_list[n-1]
    mct_ancilla = anc_qubits[:n]

    # print(f"        -> Inv CMA_Const: Subtract {k_add} mod {N} (Controlled by {len(control_list)})") # Verbose
    # Reverse the steps of controlled_modular_adder_const

    # 6_inv. Compute MSB flag copy (Forward step 3)
    multi_controlled_X(qc, control_list + [msb_qubit], flag_qubit, mct_ancilla) # Forward MCT

    # 5_inv. Compute Addition of N (Forward step 2) = Inverse of Uncompute Sub N
    controlled_adder_const_QFT_NO_MOD_dagger(qc, control_list, target_q_list, N) # Forward Subtract N = Inverse Add N

    # 4_inv. Conditionally Subtract N back (Inverse of Add N back)
    add_N_control_list = control_list + [flag_qubit]
    controlled_adder_const_QFT_NO_MOD_dagger(qc, add_N_control_list, target_q_list, N)

    # 3_inv. Uncompute MSB flag copy (Inverse step 3)
    multi_controlled_X(qc, control_list + [msb_qubit], flag_qubit, mct_ancilla) # Reverse MCT

    # 1_inv. Subtract k_add (mod 2^n) (Inverse of Add k_add)
    controlled_adder_const_QFT_NO_MOD_dagger(qc, control_list, target_q_list, k_add)

    # qc.barrier(label=f"InvCMA({k_add})") # Remove barrier


# --- Controlled Modular Multiplier Implementation ---

def controlled_modular_multiplier(qc, control_qubit, target_reg, anc_reg, k, N):
    """
    Implements |c>|y> -> |c>|y*k mod N> using the accumulator method
    with controlled modular adders for constants.
    Requires anc_reg >= 2n+1.
    """
    n = len(target_reg)
    num_anc = len(anc_reg)
    # print(f"    -- Start C-Mult by {k} mod {N} (Ctrl: {control_qubit}) --") # Verbose

    required_ancillas = 2 * n + 1
    if num_anc < required_ancillas:
         raise ValueError(f"Need at least {required_ancillas} ancillas for CMM implementation.")

    # Define lists of qubit objects
    target_q_list = [target_reg[i] for i in range(n)]
    acc_q_list = [anc_reg[i] for i in range(n)] # Use first n ancillas as accumulator
    mod_add_anc_qubits = [anc_reg[i] for i in range(n, 2*n + 1)] # Use next n+1 ancillas for adder

    # --- Implementation Steps ---
    # print(f"      ... Accumulating product ...") # Verbose
    # 1. Iterate through bits 'i' of the target register |y>
    for i in range(n):
        # 2. Calculate the value to add if y_i=1 and control_qubit=1
        val_to_add = pow(k * (1 << i), 1, N)
        if val_to_add == 0: continue

        # 3. Implement controlled modular addition of val_to_add to accumulator
        control_list = [control_qubit, target_q_list[i]]
        controlled_modular_adder_const(qc, control_list, acc_q_list, mod_add_anc_qubits, val_to_add, N)

    # 4. Swap result from accumulator (acc_q_list) to target register (target_q_list),
    #    controlled ONLY by the main control_qubit.
    # print(f"      ... Swapping result ...") # Verbose
    for i in range(n):
        ccx(qc, control_qubit, target_q_list[i], acc_q_list[i])
        ccx(qc, control_qubit, acc_q_list[i], target_q_list[i])
        ccx(qc, control_qubit, target_q_list[i], acc_q_list[i])
    # qc.barrier(label="C-SWAP") # Remove barrier

    # 5. Uncompute the additions to clear accumulator register back to |0>
    # print(f"      ... Uncomputing additions ...") # Verbose
    for i in range(n - 1, -1, -1):
        val_to_add = pow(k * (1 << i), 1, N)
        if val_to_add == 0: continue
        # Controls = [main control_qubit, original target bit target_q_list[i]]
        control_list = [control_qubit, target_q_list[i]]
        controlled_modular_adder_const_dagger(qc, control_list, acc_q_list, mod_add_anc_qubits, val_to_add, N)

    # print(f"    -- End C-Mult by {k} mod {N} --") # Verbose
    qc.barrier(label=f"C-Mult*{k}_mod{N}")


# --- Configuration ---
L_to_run = 8 # <--- N=15. CHANGE THIS VALUE (e.g., 8 for N=143)
if 4 not in semiprimes: semiprimes[4] = 15
if L_to_run not in semiprimes: print(f"Error: L={L_to_run} not found."); exit()
N = semiprimes[L_to_run]
shots = 2048

# --- Provider and Backend Setup ---
# !!! Replace placeholders with your actual credentials !!!
provider = QuantumRingsProvider(token='rings-200.TKfz0NUc5MrEIIqTbvf94Jqm7CnNQssj', name='cbjp404@leeds.ac.uk')
backend_name = "scarlet_quantum_rings"
try: backend = provider.get_backend(backend_name); print(f"\nSuccessfully obtained backend: {backend_name}")
except Exception as e: print(f"\nError obtaining backend {backend_name}: {e}"); exit()

# --- Shor's Algorithm Parameters & Qubit Calculation ---
n = L_to_run
num_target_qubits = n; num_phase_qubits = 2 * n;
num_anc_qubits = 2 * n + 1 # For CMM accumulator approach
total_qubits = num_phase_qubits + num_target_qubits + num_anc_qubits
print(f"\n--- Setting up Shor's Framework for N={N} (L={n}) ---")
print(f"  Phase qubits = {num_phase_qubits}, Target qubits = {num_target_qubits}, Ancilla qubits = {num_anc_qubits}, Total = {total_qubits}")

# Choose 'a'
a = None; factor_found_classically = None
if N > 2:
    if N == 15: a = 7; print(f"Using specific base a = {a} for N=15")
    else:
        attempts = 0
        while True:
            a = random.randint(2, N - 1); common_divisor = math.gcd(a, N)
            if common_divisor == 1: print(f"Selected random base a = {a} (coprime to N)"); break
            elif common_divisor > 1: print(f"Lucky find: gcd({a}, {N}) = {common_divisor}. Factor found classically."); factor_found_classically = common_divisor; break
            attempts += 1
            if attempts > 2*N: print("Could not find suitable 'a'."); exit()
else: print("N must be > 2."); exit()
if factor_found_classically: print(f"\nFactors: {factor_found_classically}, {N // factor_found_classically}"); exit()

# --- Define Registers ---
phase_reg = QuantumRegister(num_phase_qubits, 'phase')
target_reg = QuantumRegister(num_target_qubits, 'target')
anc_reg = AncillaRegister(num_anc_qubits, 'anc')
classical_reg = ClassicalRegister(num_phase_qubits, 'c')

# --- Create Quantum Circuit ---
qc = QuantumCircuit(phase_reg, target_reg, anc_reg, classical_reg, name=f"Shor_ModExp_N{N}_a{a}")

# --- Initialize Registers ---
print(f"\nInitializing phase register ({num_phase_qubits} qubits)..."); qc.h(phase_reg)
print(f"Initializing target register ({num_target_qubits} qubits) to |1>...");
if num_target_qubits > 0: qc.x(target_reg[0])
qc.barrier(label="Init")

# --- Modular Exponentiation via Repeated Squaring ---
print(f"Applying modular exponentiation structure for a={a}, N={N}...")
a_powers = [pow(a, 2**j, N) for j in range(num_phase_qubits)]

# Apply controlled modular multiplications
for j in range(num_phase_qubits):
    k = a_powers[j]
    if k == 1: print(f"    Skipping C-Mult by k=1 (control={phase_reg[j]})"); continue
    control_qubit = phase_reg[j]
    # Call the CMM function (attempts full implementation)
    controlled_modular_multiplier(qc, control_qubit, target_reg, anc_reg, k, N)

# --- Inverse Quantum Fourier Transform ---
print(f"\nApplying Inverse QFT on phase register ({num_phase_qubits} qubits)...")
iqft_cct(qc, phase_reg, num_phase_qubits)

# --- Measurement ---
print(f"Adding measurement to phase register ({num_phase_qubits} qubits)...")
qc.measure(phase_reg, classical_reg)
qc.barrier(label="Measure")

# --- Execute on Backend ---
print(f"\nExecuting Shor's circuit for N={N} on backend: {backend_name}...")
job = None; current_job_id = "N/A"; current_job_status = "N/A"
try:
    # print("\nCircuit QASM:\n", qc.qasm()) # Optional: Print QASM for debugging
    job = backend.run(qc, shots=shots)
    current_job_id = job.job_id; print(f"Job submitted with ID: {current_job_id}")
    print("Monitoring job status..."); job_monitor(job)
    current_job_status = job.status(); print(f"\nFinal job status for N={N}: {current_job_status}")
    if current_job_status == JobStatus.DONE:
        result = job.result(); counts = result.get_counts()
        print("\nSimulation Results (Counts - Phase Register):"); print(counts)
        plot_title = f"Shor's Results (Phase Register) for N={N}, a={a}"
        print("\nPlotting histogram...")
        if counts: plot_histogram(counts, title=plot_title)
        else: print("No counts received.")
        if counts:
            estimated_r = find_period_from_counts(counts, num_phase_qubits, N, a)
            if estimated_r:
                print(f"\n--- Attempting to find factors using period r = {estimated_r} ---")
                if estimated_r % 2 != 0: print(f" Period r={estimated_r} is odd. Try different 'a'.")
                else:
                    term = pow(a, estimated_r // 2, N)
                    if term == N - 1 or term == 1: print(f" Term a^(r/2) = {term} mod {N}. Trivial factors. Try different 'a'.")
                    else:
                        factor1 = math.gcd(term + 1, N); factor2 = math.gcd(term - 1, N)
                        print(f" Calculating gcd(a^(r/2) - 1, N) = gcd({term - 1}, {N}) = {factor1}")
                        print(f" Calculating gcd(a^(r/2) + 1, N) = gcd({term + 1}, {N}) = {factor2}")
                        found_factors = {f for f in [factor1, factor2] if f > 1 and f < N}
                        if found_factors: print(f"\n*** Non-trivial factors found: {found_factors} ***")
                        else: print(f" Found only trivial factors (1 or N). Try different 'a'.")
            else: print("\n--- Failed to find factors: Period estimation unsuccessful. ---")
        else: print("\n--- Failed to find factors: No measurement counts received. ---")
    else: print(f"Job did not complete successfully. Status: {current_job_status}")
except NotImplementedError as nie:
    print(f"\nEXECUTION HALTED: A required component is not fully implemented: {nie}")
except Exception as e:
    print(f"\nAn error occurred: {e}")
    import traceback
    traceback.print_exc() # Print full traceback for debugging
    print(f"Job ID was: {current_job_id}")
    if job:
        try: final_status_check = job.status(); print(f"Final status check: {final_status_check}")
        except Exception as status_e: print(f"Could not retrieve final status: {status_e}")
print("\nScript finished.")



Successfully imported 'semiprimes' dictionary from semiprimes.py
QuantumRingsLib version: 0.9.0

Successfully obtained backend: scarlet_quantum_rings

--- Setting up Shor's Framework for N=143 (L=8) ---
  Phase qubits = 16, Target qubits = 8, Ancilla qubits = 17, Total = 41
Selected random base a = 25 (coprime to N)

Initializing phase register (16 qubits)...
Initializing target register (8 qubits) to |1>...
Applying modular exponentiation structure for a=25, N=143...

Applying Inverse QFT on phase register (16 qubits)...
Adding measurement to phase register (16 qubits)...

Executing Shor's circuit for N=143 on backend: scarlet_quantum_rings...
Job submitted with ID: 0
Monitoring job status...
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Running
Job Runni